# Example notebook for different normalization techniques

normalization - zscore time series, zscore for post processing, run the decoder for both of them. Chose ECoG contacts, quantile just for feature norm.

Notebooks for normalization (real data; no norm, zscore, quantile transform) and check decoding performance differences



Features or raw normalization?



Normalization methods available in py_neuromodulation are:

  MEAN = "mean"

  MEDIAN = "median"

  ZSCORE = "zscore"

  ZSCORE_MEDIAN = "zscore-median"

  QUANTILE = "quantile"

  POWER = "power"

  ROBUST = "robust"

  MINMAX = "minmax"`

We are going to explore the differences between no normalization, zscoring and quantile normalization.

In pre-processing, raw normalization refers to normalizing the data with respect to the past number of "normalize_samples".

In post-processing, feature normalization refers to normalizing features with respect to the past number of "normalize_samples".

In [1]:
import os
import sys
import py_neuromodulation as nm
import xgboost
from py_neuromodulation import (
    nm_analysis,
    nm_decode,
    nm_define_nmchannels,
    nm_IO,
    nm_plots,
    nm_stats
)
from sklearn import metrics, model_selection
import json
import matplotlib.pyplot as plt
import numpy as np
import re

# change root directory of the project
SCRIPT_DIR = os.path.dirname(os.path.abspath(''))
if SCRIPT_DIR.split("/")[-1] == "py_neuromodulation":
    # this check is necessary, so we can also run the script from the root directory
    SCRIPT_DIR = os.path.join(SCRIPT_DIR, "examples")

sys.path.append(os.path.dirname(SCRIPT_DIR))

# Reload imports to get changes in other scripts
%load_ext autoreload
%autoreload 2

In [2]:
sub = "testsub"
ses = "EphysMedOff"
task = "buttonpress"
run = 0
datatype = "ieeg"

# Define run name and access paths in the BIDS format.
RUN_NAME = f"sub-{sub}_ses-{ses}_task-{task}_run-{run}"

PATH_RUN = os.path.join(
    (os.path.join(SCRIPT_DIR, "data")),
    f"sub-{sub}",
    f"ses-{ses}",
    datatype,
    RUN_NAME,
)
PATH_BIDS = os.path.join(SCRIPT_DIR, "data")


(
    raw,
    data,
    sfreq,
    line_noise,
    coord_list,
    coord_names,
) = nm_IO.read_BIDS_data(
    PATH_RUN=PATH_RUN, BIDS_PATH=PATH_BIDS, datatype=datatype
);

Extracting parameters from /home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/Code/py_neuromodulation/examples/data/sub-testsub/ses-EphysMedOff/ieeg/sub-testsub_ses-EphysMedOff_task-buttonpress_run-0_ieeg.vhdr...
Setting channel info structure...
Reading channel info from /home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/Code/py_neuromodulation/examples/data/sub-testsub/ses-EphysMedOff/ieeg/sub-testsub_ses-EphysMedOff_task-buttonpress_run-0_channels.tsv.
Reading electrode coords from /home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/Code/py_neuromodulation/examples/data/sub-testsub/ses-EphysMedOff/ieeg/sub-testsub_ses-EphysMedOff_acq-StimOff_space-mni_electrodes.tsv.


### No Normalization

In [3]:
# Provide a path for the output data. Each re-referencing method has their PATH_OUT
PATH_OUT_NONE = os.path.join(SCRIPT_DIR, "data", "derivatives", "normalization", "none")
nm_channels_None = nm_define_nmchannels.set_channels(
    ch_names=raw.ch_names,
    ch_types=raw.get_channel_types(),
    reference='default',
    bads=raw.info["bads"],
    new_names="default",
    used_types=("ecog",),  # We focus only on LFP data
    target_keywords=("SQUARED_ROTATION",),
)

stream_None = nm.Stream(
    settings=None,
    nm_channels=nm_channels_None,
    path_grids=None,
    verbose=True,
)

stream_None.set_settings_fast_compute()

# stream.reset_settings() DOESNT WORK!!!

stream_None.settings['preprocessing']['raw_normalization'] = False
stream_None.settings['postprocessing']['feature_normalization'] = False



stream_None.init_stream(
    sfreq=sfreq,
    line_noise=line_noise,
    coord_list=coord_list,
    coord_names=coord_names
)

stream_None.run(
    data=data,
    out_path_root=PATH_OUT_NONE,
    folder_name=RUN_NAME,
)

No Error occurred when testing the settings.
No data specified. Sanity checks related to the length of the signal relative to the filter order will be skipped.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 7.50 Hz
- Upper transition bandwidth: 7.50 Hz
- Filter length: 999 samples (0.999 sec)

Last batch took: 0.01 seconds
1.0 seconds of data processed
Last batch took: 0.01 seconds
1.1 seconds of data processed
Last batch took: 0.01 seconds
1.2 seconds of data processed
Last batch took: 0.01 seconds
1.3 seconds of data processed
Last batch took: 0.01 seconds
1.4 seconds of data processed
Last batch took: 0.01 seconds
1.5 seconds of data processed
Last batch took: 0.01 seconds
1.6 seconds of data processed
Last batch took: 0.01 seconds
1.7 seconds of 

 ### Zscore
 The normalized array of the current sample subtracts the previous sample mean and divides it by the previous sample standard deviation.
 #### 1. Zscore in pre-processing

In [4]:
# Provide a path for the output data. Each re-referencing method has their PATH_OUT
PATH_OUT_ZSCORE_PRE = os.path.join(SCRIPT_DIR, "data", "derivatives", "normalization", "zscore_pre")
nm_channels_zscorePre = nm_define_nmchannels.set_channels(
    ch_names=raw.ch_names,
    ch_types=raw.get_channel_types(),
    reference='default',
    bads=raw.info["bads"],
    new_names="default",
    used_types=("ecog",),  # We focus only on LFP data
    target_keywords=("SQUARED_ROTATION",),
)

stream_zscorePre = nm.Stream(
    settings=None,
    nm_channels=nm_channels_zscorePre,
    path_grids=None,
    verbose=True,
)

stream_zscorePre.set_settings_fast_compute()

# stream.reset_settings() DOESNT WORK!!!

stream_zscorePre.settings['preprocessing']['raw_normalization'] = True
stream_zscorePre.settings['postprocessing']['feature_normalization'] = False
stream_zscorePre.settings[ "raw_normalization_settings"]["normalization_method"] = 'zscore'



stream_zscorePre.init_stream(
    sfreq=sfreq,
    line_noise=line_noise,
    coord_list=coord_list,
    coord_names=coord_names
)

stream_zscorePre.run(
    data=data,
    out_path_root=PATH_OUT_ZSCORE_PRE,
    folder_name=RUN_NAME,
)

No Error occurred when testing the settings.
No data specified. Sanity checks related to the length of the signal relative to the filter order will be skipped.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 7.50 Hz
- Upper transition bandwidth: 7.50 Hz
- Filter length: 999 samples (0.999 sec)

Last batch took: 0.01 seconds
1.0 seconds of data processed
Last batch took: 0.01 seconds
1.1 seconds of data processed
Last batch took: 0.01 seconds
1.2 seconds of data processed
Last batch took: 0.01 seconds
1.3 seconds of data processed
Last batch took: 0.01 seconds
1.4 seconds of data processed
Last batch took: 0.01 seconds
1.5 seconds of data processed
Last batch took: 0.01 seconds
1.6 seconds of data processed
Last batch took: 0.01 seconds
1.7 seconds of 

#### 2. Zscore in post-processing

In [5]:
# Provide a path for the output data. Each re-referencing method has their PATH_OUT
PATH_OUT_ZSCORE_POST = os.path.join(SCRIPT_DIR, "data", "derivatives", "normalization", "zscore_post")
nm_channels_zscorePost = nm_define_nmchannels.set_channels(
    ch_names=raw.ch_names,
    ch_types=raw.get_channel_types(),
    reference='default',
    bads=raw.info["bads"],
    new_names="default",
    used_types=("ecog",),  # We focus only on LFP data
    target_keywords=("SQUARED_ROTATION",),
)

stream_zscorePost = nm.Stream(
    settings=None,
    nm_channels=nm_channels_zscorePost,
    path_grids=None,
    verbose=True,
)

stream_zscorePost.set_settings_fast_compute()

# stream.reset_settings() DOESNT WORK!!!

stream_zscorePost.settings['preprocessing']['raw_normalization'] = False
stream_zscorePost.settings['postprocessing']['feature_normalization'] = True
stream_zscorePost.settings[ "feature_normalization_settings"]["normalization_method"] = 'zscore'



stream_zscorePost.init_stream(
    sfreq=sfreq,
    line_noise=line_noise,
    coord_list=coord_list,
    coord_names=coord_names
)

stream_zscorePost.run(
    data=data,
    out_path_root=PATH_OUT_ZSCORE_POST,
    folder_name=RUN_NAME,
)

No Error occurred when testing the settings.
No data specified. Sanity checks related to the length of the signal relative to the filter order will be skipped.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 7.50 Hz
- Upper transition bandwidth: 7.50 Hz
- Filter length: 999 samples (0.999 sec)

Last batch took: 0.01 seconds
1.0 seconds of data processed
Last batch took: 0.01 seconds
1.1 seconds of data processed
Last batch took: 0.01 seconds
1.2 seconds of data processed
Last batch took: 0.01 seconds
1.3 seconds of data processed
Last batch took: 0.01 seconds
1.4 seconds of data processed
Last batch took: 0.01 seconds
1.5 seconds of data processed
Last batch took: 0.01 seconds
1.6 seconds of data processed
Last batch took: 0.01 seconds
1.7 seconds of 

### Quantile

Quantile normalization is a technique for making two distributions identical in statistical properties. To quantile-normalize a test distribution to a reference distribution of the same length, sort the test distribution and sort the reference distribution. The highest entry in the test distribution then takes the value of the highest entry in the reference distribution, the next highest entry in the reference distribution, and so on, until the test distribution is a perturbation of the reference distribution. [1]

#### References
[1] Quantile normalization, https://en.wikipedia.org/wiki/Quantile_normalization